# Average number of pairwise differences

In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.6
SparkUI available at http://hms-beagle-5466c684ff-2l8nm:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.58-3f304aae6ce2
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/hail-20201201-1248-0.2.58-3f304aae6ce2.log


In [2]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [74]:
mt = hl.read_matrix_table('/home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/mt/filtered.mt')

Calculate the allele dosage.

## Dosage difference

Calculate differences between to samples at a site as the difference in number of alternate alleles (dosage).

Confusion matrix:

|  | 0 | 1 | 2
---|---|---|---|
0  | 0  | 1  | 2
1  | 1  | 0  | 1
2  | 2  | 1  | 0


In [117]:
mt = mt.annotate_entries(DS=mt.GT[0] + mt.GT[1])

In [5]:
mt_small = mt.sample_rows(0.00001)

In [29]:
ht = mt.localize_entries("entries", "cols")
ht = ht.annotate(sum_of_pairwise_diffs = hl.sum(
        hl.range(0, hl.len(ht.entries))
            .flatmap(
                lambda i: hl.range(0, i).map(lambda j: hl.abs(ht.entries[i].DS - ht.entries[j].DS)))))
ht = ht.annotate(avg_pairwise_diff=ht.sum_of_pairwise_diffs/(hl.len(ht.entries) * (hl.len(ht.entries) - 1) / 2))

In [30]:
ht.avg_pairwise_diff.take(5)

[0.0416666679084301,
 0.11968085169792175,
 0.1560283750295639,
 0.31117022037506104,
 0.29609930515289307]

## Nucleotide diversity

In [296]:
def get_mt_allele(mt, allele):
    '''
    Replace the GT call field with the specified allele. Add the allele number to
    the column key.
    '''
    mt2 = mt.transmute_entries(GT=mt.GT[allele])
    mt2 = mt2.key_cols_by(s=mt2.s + '_' + str(allele))
    return mt2

def explode_alleles(mt):
    '''
    Split each sample into the reference and alternate allele.
    
    Input:
    mt:      MatrixTable with diallelic GT call field.

    Returns:
    MatrixTable
    '''
    
    # Get a MatrixTable with the reference allele, where the sample names are
    # appended with an "_0".
    mt_ref = get_mt_allele(mt, 0)
    # Similarly for the alternate allele.
    mt_alt = get_mt_allele(mt, 1)

    # Concatenate the samples of the two MatrixTables.
    mt_new = mt_ref.union_cols(mt_alt)
    
    return mt_new

In [322]:
mt_new = explode_alleles(mt)

In [329]:
ht = mt_new.localize_entries("entries", "cols")
ht = ht.annotate(sum_of_pairwise_diffs = hl.sum(
        hl.range(0, hl.len(ht.entries))
            .flatmap(
                lambda i: hl.range(0, i).map(lambda j: hl.abs(ht.entries[i].GT - ht.entries[j].GT)))))
ht = ht.annotate(avg_pairwise_diff=ht.sum_of_pairwise_diffs/(hl.len(ht.entries) * (hl.len(ht.entries) - 1) / 2))

In [330]:
%%time
ht.aggregate(hl.agg.mean(ht.sum_of_pairwise_diffs))

CPU times: user 29.4 ms, sys: 16.6 ms, total: 46 ms
Wall time: 5min 22s


553.3932364063428

## Average pairwise differences per gene

In [335]:
ht = ht.annotate(gene=ht.info.ANN.map(lambda x: x.split('\|')[3]))
ht = ht.annotate(gene1=ht.gene[0])

In [348]:
gene_stats = (ht.group_by(ht.gene1)
                       .aggregate(pi=hl.agg.mean(ht.avg_pairwise_diff)))

In [349]:
%%time
gene_stats.pi.summarize()

2020-12-02 13:46:29 Hail: INFO: Ordering unsorted dataset with network shuffle


Non-missing,25731 (100.00%)
Missing,0
Minimum,0.00
Maximum,1.48
Mean,0.12
Std Dev,0.07


CPU times: user 30.5 ms, sys: 7.6 ms, total: 38.1 ms
Wall time: 2min 4s


In [351]:
p = hl.plot.histogram(gene_stats.pi, range=(0, 0.26), bins=100,
                      title='Histogram of average pairwise differences per gene')
p.plot_width = 800
p.plot_height = 500
show(p)

2020-12-02 13:49:05 Hail: INFO: Ordering unsorted dataset with network shuffle


## A less stupid way of computing nucleotide diversity

We need the number of alleles which is $2 N$ where $N$ is the number of samples.

In [75]:
n_alleles = 2 * mt.count_cols()
mt = mt.annotate_rows(n_alt=mt.info.AC[0])

For a **single site**, the number of pairwise differences between sequences can be calculated as $m (N - m)$, where $m$ is the number of either allele (e.g. the reference allele). The average pairwise differences is then obtained by dividing this by the number of total comparisons.

$$
\pi = \frac{m (N - m)}{N (N - 1) / 2}
$$

In [76]:
mt = mt.annotate_rows(pi=mt.n_alt * (n_alleles - mt.n_alt) / (n_alleles * (n_alleles - 1) / 2))

Annotate the `MatrixTable` with gene names. Where there are multiple transcripts, the first one is chosen.

In [77]:
mt = mt.annotate_rows(gene=mt.info.ANN.map(lambda x: x.split('\|')[3]))
mt = mt.annotate_rows(gene1=mt.gene[0])

Group the sites by gene, and compute the average pairwise differences for each gene.

In [92]:
rows = mt.rows()
gene_stats = (rows.group_by(rows.gene1)
                       .aggregate(pi = hl.agg.mean(rows.pi),
                                 n_sites=hl.agg.count(),
                                 avg_ac=hl.agg.mean(rows.info.AC[0])))

In [79]:
gene_stats.pi.summarize()

2020-12-03 15:12:56 Hail: INFO: Ordering unsorted dataset with network shuffle


Non-missing,20200 (100.00%)
Missing,0
Minimum,0.00
Maximum,0.51
Mean,0.05
Std Dev,0.05


In [80]:
p = hl.plot.histogram(gene_stats.pi, range=(0, 0.30), bins=100,
                      title='Histogram of average pairwise differences per gene')
p.plot_width = 800
p.plot_height = 500
show(p)

2020-12-03 15:13:04 Hail: INFO: Ordering unsorted dataset with network shuffle


In [93]:
gene_stats_filtered = gene_stats.filter((gene_stats.pi>0.0205) & (gene_stats.pi < 0.021))

In [94]:
gene_stats_filtered.pi.summarize()

2020-12-03 15:17:34 Hail: INFO: Ordering unsorted dataset with network shuffle


Non-missing,6002 (100.00%)
Missing,0
Minimum,0.02
Maximum,0.02
Mean,0.02
Std Dev,0.00


In [96]:
gene_stats_filtered.n_sites.summarize()

2020-12-03 15:18:10 Hail: INFO: Ordering unsorted dataset with network shuffle


Non-missing,6002 (100.00%)
Missing,0
Minimum,1
Maximum,28
Mean,3.62
Std Dev,3.09


In [95]:
gene_stats_filtered.avg_ac.summarize()

2020-12-03 15:17:40 Hail: INFO: Ordering unsorted dataset with network shuffle


Non-missing,6002 (100.00%)
Missing,0
Minimum,1.00
Maximum,95.00
Mean,1.05
Std Dev,1.95
